In [1]:
import numpy as np
import pandas as pd
import os
import sklearn 

In [2]:
os.chdir('/Users/mayurbansal/documents/github/Dotin-Columbia-Castone-Team-Alpha-/data/Clean Data')

In [3]:
#read data
data=pd.read_csv("../Clean Data/merged_data_user_level.csv",index_col=0)

In [4]:
data.head()

,user_id,scroll count,mouse movement count,click count,User Record Count,Max Time Lapse,Total Distance,validation,Min Score.x,Min Score.y,...,no_horizontal_movement,perc_left_movement,perc_right_movement,perc_no_movement_x,movesup,movesdown,no_vertical_movement,perc_upwward_movement,perc_downward_movement,perc_no_movement_y
1,365,11,4766,228,5005,245.808,5005,False,1,5,...,2245,28.517186,26.618705,44.864109,1512,2620,872,30.215827,52.358114,17.426059
2,371,1,3900,143,4044,185.388,4044,True,1,7,...,805,41.578036,38.511007,19.910957,1327,1950,766,32.822162,48.231511,18.946327
3,373,35,5692,225,5952,200.144,5952,False,1,7,...,1429,39.438750,36.548479,24.012771,1719,2539,1693,28.885902,42.665098,28.449000
4,374,32,4666,277,4975,249.975,4975,True,3,6,...,1599,36.771210,31.081624,32.147165,1626,2169,1179,32.689988,43.606755,23.703257
5,375,70,3758,77,3905,577.872,3905,False,1,5,...,998,37.064549,37.371926,25.563525,1718,1476,710,44.006148,37.807377,18.186475


In [5]:
#fill na values with 0
data['perc_left_movement'].fillna(0,inplace=True)
data['perc_right_movement'].fillna(0,inplace=True)
data['perc_no_movement_x'].fillna(0,inplace=True)

### have 2 na values in each of these columns, replacing these with 0 for now

In [6]:
#logistic regression
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 1 to 730
Data columns (total 47 columns):
user_id                   730 non-null int64
scroll count              730 non-null int64
mouse movement count      730 non-null int64
click count               730 non-null int64
User Record Count         730 non-null int64
Max Time Lapse            730 non-null float64
Total Distance            730 non-null int64
validation                730 non-null bool
Min Score.x               730 non-null int64
Min Score.y               730 non-null int64
Min Score                 730 non-null int64
bf _votes_ 1              730 non-null int64
bf _votes_ 5              730 non-null int64
bf _votes_ 3              730 non-null int64
bs _votes_ 3              730 non-null int64
miq _votes_ 3             730 non-null int64
pgi _votes_ 4             730 non-null int64
bs _votes_ 1              730 non-null int64
bs _votes_ 4              730 non-null int64
bs _votes_ 5              730 non-null 

In [ ]:
# Split data into 'X' features and 'y' target label sets
y = data['validation']
X=data.iloc[:,np.r_[1:6,11:32,35:46]]

### for our first model we have chosen the following variables:

### -scroll count, movement count, click count, total record count
### -max time elapsed, total distance
### -bf_votes_1 etc i.e. choice of answers for each category for each question 
### -movement directions of user i.e. counts of left,right,up,down movements and percentage of each of these movements

## 70,30 split for the model

In [ ]:

# Split data set into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
# Import module for fitting
from sklearn.linear_model import LogisticRegression
# Create instance (i.e. object) of LogisticRegression
logmodel = LogisticRegression()
# Fit the model using the training data
#X_train -> parameter supplies the data features
#y_train -> parameter supplies the target labels
logmodel.fit(X_train, y_train)

In [ ]:
y_pred = logmodel.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logmodel.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


### Accuracy and other metrics are close to 75% for the first model using some features we created

In [ ]:
#Feature selection 
#Recursive feature elimination
from sklearn.feature_selection import RFE

#Select best feature 
rfe = RFE(logmodel, n_features_to_select= 10)
rfe = rfe.fit(X_train, y_train)

#Summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)
X_train.columns[rfe.support_]

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

### Using tree based classifier method for feature selection, we find following features as important:

### - How users answer some questions
### - movements right,left, down etc
### -no of records(scroll+click+moves) by user
### -total time taken by user to complete survey